In [1]:
import sys
sys.path.insert(0, '../preprocessing')
import preprocessing as pre

import numpy as np
import pandas as pd
import tensorflow as tf

/Users/jinli/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
(X_train, y_train,
 X_valid, y_valid,
 X_test, y_test) = pre.train_validate_test(r'../data/planesnet.json')

Size of Train Set is: 20480
Size of Validation Set is: 5120
Size of Test Set is: 6400


In [3]:
def reset_graph (seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [4]:
X_train_s = pre.scale (X_train, scale='constant')
X_train_s

array([[0.78039217, 0.78039217, 0.7921569 , ..., 0.9098039 , 0.9254902 ,
        0.93333334],
       [0.6627451 , 0.68235296, 0.7019608 , ..., 0.7490196 , 0.6901961 ,
        0.6156863 ],
       [0.8       , 0.8       , 0.8039216 , ..., 0.80784315, 0.7764706 ,
        0.7607843 ],
       ...,
       [0.4392157 , 0.45882353, 0.43529412, ..., 0.4392157 , 0.43529412,
        0.42745098],
       [0.8235294 , 0.8235294 , 0.83137256, ..., 0.69411767, 0.69411767,
        0.6901961 ],
       [0.68235296, 0.62352943, 0.60784316, ..., 0.627451  , 0.5882353 ,
        0.54901963]], dtype=float32)

In [5]:
X_train_s.shape

(20480, 1200)

In [78]:
# parameters
height = 20
width = 20
channels = 3

n_inputs = height * width * channels

conv1_fmaps = 64
conv1_ksize = 3
conv1_stride = 2
conv1_pad = "SAME"

conv2_fmaps = 128
conv2_ksize = 3
conv2_stride = 2
conv2_pad = "SAME"

pool3_fmaps = conv2_fmaps

n_fc1 = 64
n_outputs = 2

reset_graph()

In [79]:
with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")

In [80]:
conv1 = tf.layers.conv2d(X_reshaped, 
                         filters=conv1_fmaps, 
                         kernel_size=conv1_ksize,
                         strides=conv1_stride, 
                         padding=conv1_pad,
                         activation=tf.nn.relu, 
                         name="conv1")

with tf.name_scope('pool1'):
    pool1 = tf.nn.max_pool (conv1,
                            ksize=[1, 2, 2, 1], 
                            strides=[1, 2, 2, 1], 
                            padding="VALID")
    
lrn1 = tf.nn.local_response_normalization(pool1,)

conv2 = tf.layers.conv2d(lrn1, 
                         filters=conv2_fmaps, 
                         kernel_size=conv2_ksize,
                         strides=conv2_stride, 
                         padding=conv2_pad,
                         activation=tf.nn.relu, 
                         name="conv2")

In [81]:
with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, 
                           ksize=[1, 2, 2, 1], 
                           strides=[1, 2, 2, 1], 
                           padding="VALID")
    pool3_flat = tf.reshape(pool3, 
                            shape=[-1, pool3_fmaps])

In [82]:
with tf.name_scope("fc1"):
    # Turn drop out on for training, off for testing.
    training = tf.placeholder_with_default(False, shape=(), name='training')
    
    dropout_rate = .5 # 1 - keep_prob
    
    fc1 = tf.layers.dense(pool3_flat, 
                          n_fc1, 
                          activation=tf.nn.relu, 
                          name="fc1")
    fc1_drop = tf.layers.dropout(fc1, 
                                 dropout_rate,
                                 training=training)

In [83]:
with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, 
                             n_outputs, 
                             name="output")
    Y_proba = tf.nn.softmax(logits, 
                            name="Y_proba")

In [84]:
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

In [85]:
with tf.name_scope("eval"):
    # returns 1D tensor of booleans
    correct = tf.nn.in_top_k(logits, y, 1) 
    # turn booleans into floats and find average
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32)) 

In [86]:
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [88]:
n_epochs = 30
batch_size = 40
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train_s, y_train, batch_size):
            sess.run(training_op, feed_dict={training:True,
                                             X: X_batch, 
                                             y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

    save_path = saver.save(sess, "CNN_1/my_model_final.ckpt")

0 Batch accuracy: 0.9 Validation accuracy: 0.49140626
1 Batch accuracy: 0.925 Validation accuracy: 0.79726565
2 Batch accuracy: 0.95 Validation accuracy: 0.61992186
3 Batch accuracy: 0.95 Validation accuracy: 0.5839844
4 Batch accuracy: 1.0 Validation accuracy: 0.8984375
5 Batch accuracy: 0.975 Validation accuracy: 0.6970703
6 Batch accuracy: 0.95 Validation accuracy: 0.8226563
7 Batch accuracy: 0.975 Validation accuracy: 0.82480466
8 Batch accuracy: 1.0 Validation accuracy: 0.86992186
9 Batch accuracy: 1.0 Validation accuracy: 0.87109375
10 Batch accuracy: 0.95 Validation accuracy: 0.85683596
11 Batch accuracy: 0.975 Validation accuracy: 0.7796875
12 Batch accuracy: 0.95 Validation accuracy: 0.81503904
13 Batch accuracy: 0.975 Validation accuracy: 0.7453125
14 Batch accuracy: 0.975 Validation accuracy: 0.8283203
15 Batch accuracy: 1.0 Validation accuracy: 0.83496094
16 Batch accuracy: 0.95 Validation accuracy: 0.83125
17 Batch accuracy: 1.0 Validation accuracy: 0.78027344
18 Batch acc

In [67]:
# this is actually annoying to configure: i'll just use keras bc its easier 
# and i don't have to rename the graphs names everytime I want to add something
# keras is also a lot more user friendly

49.0

In [ ]:
64000/64

In [ ]:
6464/64